In [1]:
import numpy as np
%reload_ext autoreload
%autoreload 2

# API Requests
import requests

# Request Wrangling
import os
from os.path import basename, exists
import json

# Data wrangling
import pandas as pd

# Data visualization
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

# Machine learning model
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.pipeline import Pipeline, make_pipeline

# Check work
from sklearn.utils.validation import check_is_fitted

from scryfall_api import ScryfallAPI, ScryfallDataWrangler

# Supress warnings
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)

In [9]:
scryfall = ScryfallAPI()

print(f"API Response type: {type(scryfall.response)}")
print(f"API Request status code: {scryfall.response.status_code}")

API Response type: <class 'requests.models.Response'>
API Request status code: 200


In [10]:
scryfall.get_dataset()

{'success': True,
 'message': "Existing file 'all-cards-20221223221718.json' is the most updated version."}

In [12]:
df = scryfall.wrangler.wrangle()

print(f"df shape: {df.shape}")
print()
print(df.info())
print()
df.head()

df shape: (26378, 75)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26378 entries, 0 to 391492
Data columns (total 75 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   object             26378 non-null  object        
 1   id                 26378 non-null  object        
 2   oracle_id          26378 non-null  object        
 3   multiverse_ids     26378 non-null  object        
 4   mtgo_id            20921 non-null  float64       
 5   mtgo_foil_id       14378 non-null  float64       
 6   tcgplayer_id       26148 non-null  float64       
 7   cardmarket_id      25952 non-null  float64       
 8   name               26378 non-null  object        
 9   lang               26378 non-null  object        
 10  released_at        26378 non-null  datetime64[ns]
 11  uri                26378 non-null  object        
 12  scryfall_uri       26378 non-null  object        
 13  layout             26378 non-null  ob

,object,id,oracle_id,multiverse_ids,mtgo_id,mtgo_foil_id,tcgplayer_id,cardmarket_id,name,lang,...,n_types,is_artifact,is_creature,is_enchantment,is_instant,is_land,is_planeswalker,is_sorcery,is_tribal,price_usd
0,card,0000579f-7b35-4ed3-b44c-db2a538066fe,44623693-51d6-49ad-8cd7-140505caf02f,[109722],25527.0,25528.0,14240.0,13850.0,Fury Sliver,en,...,1,False,True,False,False,False,False,False,False,0.36
1,card,00006596-1166-4a79-8443-ca9f82e6db4e,8ae3562f-28b7-4462-96ed-be0cf7052ccc,[189637],34586.0,34587.0,33347.0,21851.0,Kor Outfitter,en,...,1,False,True,False,False,False,False,False,False,0.08
4,card,0000cd57-91fe-411f-b798-646e965eec37,9f0d82ae-38bf-45d8-8cda-982b6ead1d72,[435231],65170.0,65171.0,145764.0,301766.0,Siren Lookout,en,...,1,False,True,False,False,False,False,False,False,0.07
17,card,0001f1ef-b957-4a55-b47f-14839cdbab6f,ef027846-be81-4959-a6b5-56bd01b1e68a,[472997],78170.0,NaN,198861.0,400134.0,Venerable Knight,en,...,1,False,True,False,False,False,False,False,False,0.59
22,card,0002ab72-834b-4c81-82b1-0d2760ea96b0,645b5784-a6f7-4cf3-966a-e1a51420b96b,[488632],81171.0,NaN,215418.0,467859.0,Mystic Skyfish,en,...,1,False,True,False,False,False,False,False,False,0.08


In [ ]:
df["price_usd"] = pd.json_normalize(df["prices"])["usd"]
df.head()

# Machine Learning Price Predictor model for MTG cards

---
## 0. Untap - Project info

#### In this project, the goal is to take

#### Set restrictions:
#### <font color = "Green"> - Only non-online cards and sets will be analysed
#### - Sets that are comprised of masterpieces, Secret Lair Drops, Friday Night Magic Cards, Judge Gift promos and other exclusive, high-end or niche cards will not be analysed
#### <font color = "Green"> - Only cards that are not part of an "Un-" set will be analysed
#### - Retail price as specified by TCGPlayer
#### - Only the price of each card at the day of its respective set or product launch will be regarded
#### <font color = "Green"> - Only cards that aren't reprints will be analysed
#### <font color = "Green"> - Only cards that aren't tokens will be analysed
#### - Cards that are of the type Conspiracy, Emblem, Hero, Phenomenon, Plane, Scheme or Vanguard won't be analysed.
#### <font color = "Green"> - Basic Lands won't be analysed
#### <font color = "Green"> - Cards on The Reserved List won't be analysed
#### <font color = "Green"> - Oversized cards won't be analysed



---
## 1. Upkeep - Importing the Libraries

---
## 2. Draw - Importing the Data

---
## 3. First Main Phase - Treating df